In [1]:
import os
import sys
import zipfile
import xml.etree.ElementTree as ET
#from xml.dom import minidom

from sqlalchemy import create_engine
import psycopg2

import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../')
%matplotlib inline
#from envir import config

## Parse XML File & Convert to Pandas DataFrame

In [2]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = ET.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [3]:
df_cols = ["ROW_ID", "ESTABLISHMENT_ID", "INSPECTION_ID", "ESTABLISHMENT_NAME", "ESTABLISHMENTTYPE",
           "ESTABLISHMENT_ADDRESS","LATITUDE","LONGITUDE", "ESTABLISHMENT_STATUS", "MINIMUM_INSPECTIONS_PERYEAR",
          "INFRACTION_DETAILS", "INSPECTION_DATE", "SEVERITY", "ACTION", "COURT_OUTCOME", "AMOUNT_FINED"]

In [4]:
df = parse_XML("dinesafe.xml", df_cols)

In [5]:
df.shape

(90361, 16)

In [6]:
df.head()

,ROW_ID,ESTABLISHMENT_ID,INSPECTION_ID,ESTABLISHMENT_NAME,ESTABLISHMENTTYPE,ESTABLISHMENT_ADDRESS,LATITUDE,LONGITUDE,ESTABLISHMENT_STATUS,MINIMUM_INSPECTIONS_PERYEAR,INFRACTION_DETAILS,INSPECTION_DATE,SEVERITY,ACTION,COURT_OUTCOME,AMOUNT_FINED
0,1,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,NA - Not Applicable,Notice to Comply,,
1,2,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash equipment,2018-06-20,M - Minor,Notice to Comply,,
2,3,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash surfaces in rooms,2018-06-20,M - Minor,Notice to Comply,,
3,4,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operate food premise - equipment not arranged ...,2018-08-07,M - Minor,Notice to Comply,,
4,5,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Use handwashing station other than for handwas...,2018-08-07,S - Significant,Corrected During Inspection,,


## DataFrame Investigation

In [7]:
df.columns

Index(['ROW_ID', 'ESTABLISHMENT_ID', 'INSPECTION_ID', 'ESTABLISHMENT_NAME',
       'ESTABLISHMENTTYPE', 'ESTABLISHMENT_ADDRESS', 'LATITUDE', 'LONGITUDE',
       'ESTABLISHMENT_STATUS', 'MINIMUM_INSPECTIONS_PERYEAR',
       'INFRACTION_DETAILS', 'INSPECTION_DATE', 'SEVERITY', 'ACTION',
       'COURT_OUTCOME', 'AMOUNT_FINED'],
      dtype='object')

In [8]:
# Make column names lowercase

df.columns = map(str.lower, df.columns)

In [9]:
df.head()

,row_id,establishment_id,inspection_id,establishment_name,establishmenttype,establishment_address,latitude,longitude,establishment_status,minimum_inspections_peryear,infraction_details,inspection_date,severity,action,court_outcome,amount_fined
0,1,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,NA - Not Applicable,Notice to Comply,,
1,2,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash equipment,2018-06-20,M - Minor,Notice to Comply,,
2,3,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash surfaces in rooms,2018-06-20,M - Minor,Notice to Comply,,
3,4,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operate food premise - equipment not arranged ...,2018-08-07,M - Minor,Notice to Comply,,
4,5,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Use handwashing station other than for handwas...,2018-08-07,S - Significant,Corrected During Inspection,,


In [10]:
# Number of unique inspections 

inspections = list(df['inspection_id'])
unique_inspections = set(inspections)
print(len(unique_inspections))

56741


In [11]:
df['establishment_status'].unique()

array(['Pass', 'Conditional Pass', 'Closed'], dtype=object)

In [12]:
df['severity'].unique()

array(['NA - Not Applicable', 'M - Minor', 'S - Significant',
       'C - Crucial', None], dtype=object)

In [13]:
df['establishmenttype'].unique()

array(['Food Take Out', 'Restaurant', 'Cafeteria', 'Commissary',
       'Private Club', 'Child Care - Catered',
       'Food Store (Convenience / Variety)',
       'Child Care - Food Preparation', 'Food Depot', 'Food Court Vendor',
       'Serving Kitchen', 'Banquet Facility', 'Butcher Shop',
       'Food Processing Plant', 'Bakery', 'Supermarket',
       'Other Educational Facility Food Services',
       'Cafeteria - Public Access', 'Nursing Home / Home for the Aged',
       'Ice Cream / Yogurt Vendors', 'Retirement Homes(Licensed)',
       'Institutional Food Service', 'Secondary School Food Services',
       'Fish Shop', 'School Nourishment Program', 'Food Caterer',
       'Cocktail Bar / Beverage Room', 'Bake Shop',
       'Boarding / Lodging Home - Kitchen', 'Flea Market',
       'College/University Food services',
       'Hospitals & Health Facilities', 'Refreshment Stand (Stationary)',
       'Meat Processing Plant', 'Community Kitchen Meal Program',
       'Church Banquet Facil

In [14]:
city = []

for i in range(90361):
    name = 'toronto'
    city.append(name)

In [15]:
df['city'] = city

In [16]:
df.head()

,row_id,establishment_id,inspection_id,establishment_name,establishmenttype,establishment_address,latitude,longitude,establishment_status,minimum_inspections_peryear,infraction_details,inspection_date,severity,action,court_outcome,amount_fined,city
0,1,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,NA - Not Applicable,Notice to Comply,,,toronto
1,2,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash equipment,2018-06-20,M - Minor,Notice to Comply,,,toronto
2,3,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash surfaces in rooms,2018-06-20,M - Minor,Notice to Comply,,,toronto
3,4,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operate food premise - equipment not arranged ...,2018-08-07,M - Minor,Notice to Comply,,,toronto
4,5,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Use handwashing station other than for handwas...,2018-08-07,S - Significant,Corrected During Inspection,,,toronto


In [17]:
df.rename(columns={'establishment_name':'name', 'establishmenttype': 'establishment_type',
                  'establishment_address':'address', 'establishment_status': 'inspection_result'}, inplace=True)

In [18]:
df.head()

,row_id,establishment_id,inspection_id,name,establishment_type,address,latitude,longitude,inspection_result,minimum_inspections_peryear,infraction_details,inspection_date,severity,action,court_outcome,amount_fined,city
0,1,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,NA - Not Applicable,Notice to Comply,,,toronto
1,2,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash equipment,2018-06-20,M - Minor,Notice to Comply,,,toronto
2,3,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash surfaces in rooms,2018-06-20,M - Minor,Notice to Comply,,,toronto
3,4,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operate food premise - equipment not arranged ...,2018-08-07,M - Minor,Notice to Comply,,,toronto
4,5,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Use handwashing station other than for handwas...,2018-08-07,S - Significant,Corrected During Inspection,,,toronto


In [19]:
minor = []
not_applicable = []
significant = []
crucial = []

for i in df['severity']:
    if i == 'M - Minor':
        minor.append(1)
    else:
        minor.append(0)
        
for i in df['severity']:
    if i == 'NA - Not Applicable':
        not_applicable.append(1)
    else:
        not_applicable.append(0)        
        
for i in df['severity']:
    if i == 'C - Crucial':
        crucial.append(1)
    else:
        crucial.append(0)
        
for i in df['severity']:
    if i == 'S - Significant':
        significant.append(1)
    else:
        significant.append(0)

In [20]:
df['minor'] = minor
df['significant'] = significant
df['crucial'] = crucial

In [21]:
df.head()

,row_id,establishment_id,inspection_id,name,establishment_type,address,latitude,longitude,inspection_result,minimum_inspections_peryear,infraction_details,inspection_date,severity,action,court_outcome,amount_fined,city,minor,significant,crucial
0,1,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,NA - Not Applicable,Notice to Comply,,,toronto,0,0,0
1,2,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash equipment,2018-06-20,M - Minor,Notice to Comply,,,toronto,1,0,0
2,3,1222579,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operator fail to properly wash surfaces in rooms,2018-06-20,M - Minor,Notice to Comply,,,toronto,1,0,0
3,4,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Operate food premise - equipment not arranged ...,2018-08-07,M - Minor,Notice to Comply,,,toronto,1,0,0
4,5,1222579,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,2,Use handwashing station other than for handwas...,2018-08-07,S - Significant,Corrected During Inspection,,,toronto,0,1,0


In [22]:
df.drop(['establishment_id', 'minimum_inspections_peryear','action',
        'court_outcome','amount_fined', 'severity'],axis=1, inplace=True)

In [23]:
df.head()

,row_id,inspection_id,name,establishment_type,address,latitude,longitude,inspection_result,infraction_details,inspection_date,city,minor,significant,crucial
0,1,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,Fail to Hold a Valid Food Handler's Certificat...,2018-06-20,toronto,0,0,0
1,2,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,Operator fail to properly wash equipment,2018-06-20,toronto,1,0,0
2,3,104246429,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,Operator fail to properly wash surfaces in rooms,2018-06-20,toronto,1,0,0
3,4,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,Operate food premise - equipment not arranged ...,2018-08-07,toronto,1,0,0
4,5,104277664,SAI-LILA KHAMAN DHOKLA HOUSE,Food Take Out,870 MARKHAM RD,43.76797956,-79.229029122,Pass,Use handwashing station other than for handwas...,2018-08-07,toronto,0,1,0


In [24]:
df.fillna('', inplace=True)

In [25]:
df_inspections = df.groupby(['inspection_id', 'name', 'establishment_type', 'address',
                             'latitude', 'longitude','inspection_result','inspection_date',
                             'city'])['infraction_details'].\
                 agg({'infraction_details': ','.join, 'minor': 'count', 'significant':'count', 'crucial':'count'}).\
                 reset_index()

//anaconda3/envs/geo_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  after removing the cwd from sys.path.


In [26]:
df_inspections.drop('inspection_id', axis=1, inplace=True)
df_inspections.rename(columns={'infraction_details':'violations_desc'}, inplace=True)

df_inspections.head()

,name,establishment_type,address,latitude,longitude,inspection_result,inspection_date,city,violations_desc,minor,significant,crucial
0,SHARIF A-708,Hot Dog Cart,135 ST GEORGE ST,43.667572,-79.39984,Pass,2017-11-02,toronto,,1,1,1
1,ALLAN'S PASTRY SHOP,Bakery,3447 KENNEDY RD,43.821248816,-79.3045942709,Pass,2019-08-06,toronto,,1,1,1
2,TUKLEI INTERNATIONAL TRADING LTD,Locker Plant,440 BRIMLEY RD,43.7310022075,-79.246021516,Pass,2018-01-31,toronto,,1,1,1
3,AGNO FOOD & DAIRY PRODUCTS,Food Depot,80 BARBADOS BLVD,43.7348573853,-79.2413866044,Pass,2017-12-11,toronto,,1,1,1
4,FRONTIER FRESH,Food Depot,489 BRIMLEY RD,43.7340167008,-79.2452814486,Pass,2017-12-29,toronto,Operator fail to provide washroom supplies,1,1,1


In [ ]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect()

In [ ]:
df_inspections.to_sql('toronto_inspections_cleaned', engine, schema='clean', index=False)